# Fill empty (COALESCE/IFNULL)

Sometimes tables contain empty values. But in some cases it's convenient to replace such values with a specific value. So in this section I'll show you some features of databases that can help you to do this.

The next cell creates a docker container with postgres, which will be used for the experiments on this page.

In [2]:
%%bash
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres \
    --name fill_empty \
    postgres:15.4 &> /dev/null
sleep 5
docker exec -i fill_empty psql -U postgres -d postgres

CREATE TABLE tab(
    var1 TEXT,
    var2 TEXT
);
INSERT INTO tab (var1, var2) VALUES
(NULL, 'world'),
('hello', NULL),
(NULL, 'test line');

CREATE TABLE
INSERT 0 3


Initial state of the table used for the experiment.

In [3]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres
SELECT * FROM tab;

 var1  |   var2    
-------+-----------
       | world
 hello | 
       | test line
(3 rows)



**Note** Don't forget to close the container when you have finished playing with the examples.

In [1]:
!docker stop fill_empty

fill_empty


## In query

It's a very common case when empty values have business sense, but in query results you need to replace them. So you can write `SELECT` to replace `NULL` in query results, this section is focused on it.

### Postgres (COALESCE)

In postgres you have to use the `COALENCE` function. Use the syntax `COALESCE(<column name>, <value to replace>)`. So in the following cell such syntax is used to replace empty values.

In [4]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres

SELECT 
    COALESCE(var1, 'NAN text') var1,
    COALESCE(var2, 'NAN text') var2
FROM tab;

   var1   |   var2    
----------+-----------
 NAN text | world
 hello    | NAN text
 NAN text | test line
(3 rows)



### Alternative (IFNULL)

IFNULL is an alternative function that can be used to replace empty values in query statements.

It's not supported by postgres. So to make sure, I'll show you in the following cell that there will be an error if you try to use it in postgres.

In [5]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres
SELECT 
    IFNULL(var1, 'NAN text') var1,
    IFNULL(var2, 'NAN text') var2
FROM tab;

ERROR:  function ifnull(text, unknown) does not exist
LINE 2:     IFNULL(var1, 'NAN text') var1,
            ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


So I will use sqlite base to demonstrate how IFNULL works. In the following example is the process of creating sample database.

In [13]:
%%bash
sqlite3 fill_empty_files/dump.sql

CREATE TABLE IF NOT EXISTS tab (
    var1 TEXT,
    var2 TEXT
);
DELETE FROM tab;
INSERT INTO tab (var1, var2) VALUES
(NULL, 'world'),
('hello', NULL),
(NULL, 'test line');

Here is what the query to the example database looks like, without any of the tricks involved in replacing values.

In [16]:
%%bash
sqlite3 fill_empty_files/dump.sql

SELECT * FROM tab;

|world
hello|
|test line


The syntax is exactly the same as for `COALESCE`: `IFNULL(<column name>, <value to replace>)`.

Actually, sqlite supports both `COALESCE` and `IFNULL`, so in the following example both have been used to replace empty values in different columns.

In [15]:
%%bash
sqlite3 fill_empty_files/dump.sql

SELECT 
    IFNULL(var1, 'replaced by IFNULL') var1,
    COALESCE(var2, 'relaced by COALESCE') var2
FROM tab;

replaced by IFNULL|world
hello|relaced by COALESCE
replaced by IFNULL|test line


## Change base

Sometimes empty values in database appear as errors or business sense of data table can be selected. So you can face with case when you need to replace all values in the database, not only in the results of the specific query.

For such purpose you can use syntax:

```
UPDATE <table_name>
SET <column name> = <replace value>
WHERE <column name> IS NULL;
```

The following cell uses such syntax to replace some empty values.

In [17]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres

UPDATE tab
SET var1 = 'replaced'
WHERE var1 IS NULL;

UPDATE 2


Let's see the result of the transformation.

In [18]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres
SELECT * FROM tab;

   var1   |   var2    
----------+-----------
 hello    | 
 replaced | world
 replaced | test line
(3 rows)

